In [1]:
import numpy as np
import os
import csv
import runtests
import makegraphs

# runtests testing

In [2]:
def return_one(startdisk, zfsname, csvfilename, ip, skip_num=0):
    with open(csvfilename) as csvfile:
        reader = csv.reader(csvfile, delimiter=",")
        next(reader)
        for _ in range(skip_num):
            next(reader)
        row = next(reader)
        testname, numberdisks, raidmode, raid0, ashift, compression, recordsize, atime, filesize, benchmark, runtime, blocksizes, iodepths, numjobs = row
        print (testname, numberdisks, raidmode, raid0, ashift, compression, recordsize, atime, filesize, benchmark, runtime, blocksizes, iodepths, numjobs)
        print ("++++++++++++++++++++++++++++")
        print (testname)
        print ("++++++++++++++++++++++++++++")
        numberdisks = int(numberdisks)
        raidmode = int(raidmode)
        raid0 = int(raid0)
        return runtests.make_commands(startdisk, numberdisks, zfsname, raidmode, raid0, ashift, compression, recordsize, atime, testname, ip, filesize, benchmark, runtime, blocksizes, iodepths, numjobs)

In [34]:
cmds = return_one("c", "tank", "temptestconfig.csv", "192.168.169.207", skip_num=0)

onedisk 1 0 0 12  128k on 100g FALSE NA NA NA NA
++++++++++++++++++++++++++++
onedisk
++++++++++++++++++++++++++++


fio --directory=/tank --direct=1 --bs=32m --ioengine=psync --iodepth=1 --numjobs=32 --filesize=100g --group_reporting --name=throughput-test --eta-newline=1 --rw=write >> test_zfs_write.txt
fio --directory=/tank --direct=1 --bs=32m --ioengine=psync --iodepth=1 --numjobs=32 --filesize=100g --group_reporting --name=throughput-test --eta-newline=1 --rw=read --readonly >> test_zfs_read.txt

In [35]:
for cmd in cmds:
    print(cmd)

mkdir onedisk
echo -------------------------------
echo make zfs
echo -------------------------------
zpool create tank -o ashift=12  sdc
zfs set recordsize=128k atime=on tank
echo -------------------------------
echo test zfs
echo -------------------------------
fio --directory=/tank --direct=1 --bs=32m --ioengine=psync --iodepth=64 --filesize=100g --group_reporting --name=throughput-test --eta-newline=1 --rw=write >> onedisk/zfs_write.txt
fio --filename=/tank/throughput-test.0.0 --direct=1 --bs=32m --ioengine=psync --iodepth=64 --filesize=100g --group_reporting --name=throughput-test --eta-newline=1--rw=read --readonly >> onedisk/zfs_read.txt
rm /tank/throughput-test.0.0
echo -------------------------------
echo make lustre
echo -------------------------------
mkfs.lustre --backfstype=zfs --fsname=test --index=0 --reformat --mgs --mdt mgsmdt/mgsmdt
mkfs.lustre --backfstype=zfs --fsname=test --index=0 --ost --mgsnode=192.168.169.207@tcp tank/ost0
mount -t lustre mgsmdt/mgsmdt /mnt/mgs

fio --directory=/tank --direct=1 --rw=write --bs=32m --ioengine=libaio --iodepth=64 --filesize=30g --numjobs=2 --group_reporting --name=throughput-test --eta-newline=1

./bench_fio --target /tank --type directory -o onedisk_fioplot -b 16m 32m 64m -m write --loops 1 --iodepth 1 2 --numjobs 1 2 --ss iops:10% --ss-ramp 10 --ss-dur 30 --runtime 300 --size 20g

./fio_plot/fio_plot -T "title" -r write -g -t bw -d 1 2 -n 1 2 -i benchmark_script/onedisk_fioplot/tank/32m/

./fio-plot/fio_plot/fio_plot -T "bandwidth of write on lustre raidz2(8 drives) given iodepth and numjobs 32m" -i z2_one_8_single_lustre/lustre/16m -g -r write -t bw -d 1 2 4 -n 1 4 8
./fio-plot/fio_plot/fio_plot -T "bandwidth of write on zfs 60 (2x8 drives) given iodepth and numjobs 64m" -i z2_one_8_single_zfs/tank/64m -g -r write -t bw -d 1 2 4 -n 1 4 8
./fio-plot/fio_plot/fio_plot -T "bandwidth of read on zfs raidz2(8 drives) given iodepth and numjobs m" -i z2_one_8_single_zfs/tank/16m -g -r read -t bw -d 1 2 4 -n 1 4 8


# makegraphs testing

In [28]:
# skip_num = 11
# # TODO: stop to param
# # set stop to 0 if you don't need it
# stop = 0
# cmds = makegraphs.make_commands("config.csv", skip_num)

In [27]:
def make_commands_graph_one(csvfilename, skip_num):
    cmds = []
    with open(csvfilename) as csvfile:
        reader = csv.reader(csvfile, delimiter=",")
        next(reader)
        for _ in range(skip_num):
            next(reader)
        stop_counter = 0
        row = next(reader)
        testname, numberdisks, raidmode, raid0, ashift, compression, recordsize, atime, filesize, benchmark, runtime, blocksizes, iodepths, numjobs = row
        rws = ["read", "write"]
        fss = ["zfs", "lustre"]
        blocksizes = blocksizes.split()
        # print (blocksizes)
        graphdir = "graph_" + testname
        cmds.append ("mkdir " + graphdir)
        for bs in blocksizes:
            blockdir = graphdir +"/"+bs
            cmds.append ("mkdir "+ blockdir)
            for fs in fss:
                for rw in rws:
                    plot_cmd, title = makegraphs.make_plot_cmd (rw, fs, bs, iodepths, numjobs, testname, recordsize)
                    title = title.replace(" ", "-")
                    cmds.append (plot_cmd)
                    cmds.append ("mv " + title + '* ' + blockdir)
    return cmds

In [33]:
cmds = make_commands_graph_one ("config.csv", skip_num=19)
for cmd in cmds:
    print (cmd)

mkdir graph_z2_three_8_single_1m
mkdir graph_z2_three_8_single_1m/32m
./fio-plot/fio_plot/fio_plot -T "bandwidth of read on zfs z2_three_8_single_1m 32m1m" -i z2_three_8_single_1m/lustre/32m -g -r read -t bw -d 1 2 4 8 16 32 -n 1 2 4 8 16 32 --disable-fio-version
mv bandwidth-of-read-on-zfs-z2_three_8_single_1m-32m1m* graph_z2_three_8_single_1m/32m
./fio-plot/fio_plot/fio_plot -T "bandwidth of write on zfs z2_three_8_single_1m 32m1m" -i z2_three_8_single_1m/lustre/32m -g -r write -t bw -d 1 2 4 8 16 32 -n 1 2 4 8 16 32 --disable-fio-version
mv bandwidth-of-write-on-zfs-z2_three_8_single_1m-32m1m* graph_z2_three_8_single_1m/32m
./fio-plot/fio_plot/fio_plot -T "bandwidth of read on lustre z2_three_8_single_1m 32m1m" -i z2_three_8_single_1m/tank/32m -g -r read -t bw -d 1 2 4 8 16 32 -n 1 2 4 8 16 32 --disable-fio-version
mv bandwidth-of-read-on-lustre-z2_three_8_single_1m-32m1m* graph_z2_three_8_single_1m/32m
./fio-plot/fio_plot/fio_plot -T "bandwidth of write on lustre z2_three_8_single_